In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import io

arquivos_csv = {
    'customers': 'Customer_semicolon.csv',
    'campaigns': 'Campaign_semicolon.csv',
    'orders': 'Order_semicolon.csv',
    'campaign_sends': 'CampaignQueue_semicolon.csv'
}

dataframes = {}

for nome, arquivo in arquivos_csv.items():
    try:
        df = pd.read_csv(arquivo, sep=';')
        dataframes[nome] = df
    except FileNotFoundError:
        print(f'{arquivo} não encontrado.')

df_customers = dataframes.get('customers', pd.DataFrame())
df_orders = dataframes.get('orders', pd.DataFrame())
df_campaigns = dataframes.get('campaigns', pd.DataFrame())
df_campaign_sends = dataframes.get('campaign_sends', pd.DataFrame())

In [2]:
# TABELA DE PEDIDOS(Orders)

df_orders_cleaned = df_orders.dropna(subset=['createdAt']).copy()
# Converte a data, especificando o formato brasileiro (dayfirst)
df_orders_cleaned['createdAt'] = pd.to_datetime(df_orders_cleaned['createdAt'], dayfirst=True)

# TABELA DE CLIENTES(Customers)

df_customers['gender'] = df_customers['gender'].fillna('Não informado')
df_customers['gender'] = df_customers['gender'].replace({
    'M': 'Masculino',
    'F': 'Feminino',
    'O': 'Outro'
})

print("\n--- Contagem de Gênero (df_customers) ---")
print(df_customers['gender'].value_counts())


--- Contagem de Gênero (df_customers) ---
gender
Outro            263
Não informado    259
Feminino         244
Masculino        234
Name: count, dtype: int64


In [3]:
# CÁLCULO DE RECÊNCIA, FREQUÊNCIA E VALOR (RFM)

status_validos = ['CONCLUDED', 'DISPATCHED']
df_orders_rfm = df_orders_cleaned[df_orders_cleaned['status'].isin(status_validos)].copy()

# snapshot_date definido como um dia após a última compra registrada
snapshot_date = df_orders_rfm['createdAt'].max() + dt.timedelta(days=1)

rfm_data = df_orders_rfm.groupby('customer').agg(
    Recencia=('createdAt', lambda x: (snapshot_date - x.max()).days),
    Frequencia=('id', 'count'),
    ValorMonetario=('totalAmount', 'sum')
).reset_index()

In [4]:
# SEGMENTAÇÃO RFM E TICKET MÉDIO

# Recência: Menor é melhor
rfm_data['R_Quartil'] = pd.qcut(rfm_data['Recencia'], 4, labels=[1, 2, 3, 4]).astype(int)
# Frequência: Maior é melhor (usamos rank(method='first') para evitar valores duplicados nos quartis)
rfm_data['F_Quartil'] = pd.qcut(rfm_data['Frequencia'].rank(method='first'), 4, labels=[4, 3, 2, 1]).astype(int)
# Monetário: Maior é melhor
rfm_data['M_Quartil'] = pd.qcut(rfm_data['ValorMonetario'], 4, labels=[4, 3, 2, 1]).astype(int)

rfm_data['RFM_Score'] = rfm_data['R_Quartil'].astype(str) + rfm_data['F_Quartil'].astype(str) + rfm_data['M_Quartil'].astype(str)

# 1 é bom, 4 é ruim
condicoes_segmento = [
    (rfm_data['RFM_Score'] == '111'), # R=1, F=1, M=1
    (rfm_data['R_Quartil'] <= 2) & (rfm_data['F_Quartil'] <= 2), # R e F bons (Leais)
    (rfm_data['R_Quartil'] >= 3) & (rfm_data['F_Quartil'] <= 2), # R ruim, F bom (Em Risco)
    (rfm_data['R_Quartil'] <= 2) & (rfm_data['F_Quartil'] >= 3), # R bom, F ruim (Novos/Ocasionais)
    (rfm_data['R_Quartil'] >= 3) & (rfm_data['F_Quartil'] >= 3)  # R e F ruins (Perdidos/Baixo Valor)
]

rotulos_segmento = [
    'Clientes de Ouro',
    'Clientes Leais',
    'Em Risco (Leais)',
    'Novos/Ocasionais',
    'Perdidos/Baixo Valor'
]

rfm_data['Segmento_RFM'] = np.select(condicoes_segmento, rotulos_segmento, default='Regular')

# Cálculo do Ticket Médio por Cliente
rfm_data['TicketMedio'] = rfm_data['ValorMonetario'] / rfm_data['Frequencia']

print("--- Amostra RFM com Segmentação (Refinada) e Ticket Médio ---")
print(rfm_data.head())
print("\n--- Contagem por Segmento RFM (Refinado) ---")
print(rfm_data['Segmento_RFM'].value_counts())

--- Amostra RFM com Segmentação (Refinada) e Ticket Médio ---
   customer  Recencia  Frequencia  ValorMonetario  R_Quartil  F_Quartil  \
0         1        82           1           28.04          2          4   
1         2       267           2           97.78          4          2   
2         3        97           3          188.98          2          1   
3         5        31           1           77.81          1          4   
4         7         8           1           89.20          1          4   

   M_Quartil RFM_Score      Segmento_RFM  TicketMedio  
0          4       244  Novos/Ocasionais    28.040000  
1          2       422  Em Risco (Leais)    48.890000  
2          1       211    Clientes Leais    62.993333  
3          2       142  Novos/Ocasionais    77.810000  
4          2       142  Novos/Ocasionais    89.200000  

--- Contagem por Segmento RFM (Refinado) ---
Segmento_RFM
Perdidos/Baixo Valor    141
Novos/Ocasionais        111
Em Risco (Leais)        111
Clientes

In [5]:
# CICLO DE VIDA

def classificar_ciclo_vida_refinado(row):
    frequencia = row['Frequencia']
    recencia = row['Recencia']

    if frequencia == 1:
        if recencia <= 90: # Novo cliente que comprou recentemente
            return 'Novo'
        else: # Cliente que comprou uma vez, há muito tempo
            return 'Perdido (One-timer)'
    elif frequencia > 1:
        if recencia <= 60: # Cliente recorrente e ativo
            return 'Ativo'
        elif recencia <= 120: # Cliente recorrente em risco
            return 'Em Risco'
        else: # Cliente recorrente que não compra há mais de 120 dias
            return 'Perdido (Ex-Ativo)'
    return 'Indefinido' # Não deve acontecer em rfm_data

rfm_data['CicloDeVida'] = rfm_data.apply(classificar_ciclo_vida_refinado, axis=1)

print("--- Contagem por Ciclo de Vida (Refinado) ---")
print(rfm_data['CicloDeVida'].value_counts())

--- Contagem por Ciclo de Vida (Refinado) ---
CicloDeVida
Perdido (One-timer)    278
Novo                    96
Ativo                   54
Perdido (Ex-Ativo)      47
Em Risco                29
Name: count, dtype: int64


In [6]:
# Juntamos a base de clientes com os dados derivados de RFM
df_customers_final = df_customers.merge(
    rfm_data,
    left_on='id',
    right_on='customer',
    how='left'
)

# Clientes que não estão na tabela RFM (nunca fizeram um pedido válido) são classificados como 'Inativo'
df_customers_final['CicloDeVida'] = df_customers_final['CicloDeVida'].fillna('Inativo')

# Removemos a coluna 'customer' duplicada após o merge
if 'customer' in df_customers_final.columns:
    df_customers_final = df_customers_final.drop(columns=['customer'])

print("--- Amostra Tabela Final de Clientes Enriquecida ---")
print(df_customers_final.head())

--- Amostra Tabela Final de Clientes Enriquecida ---
   id              name               taxId         gender dateOfBirth  \
0   1   Fernanda Duarte      207.463.819-13          Outro  05/10/1972   
1   2     Matheus Jesus  46.792.503/0001-48      Masculino  19/03/1962   
2   3      João da Mota      594.173.682-73       Feminino  20/10/1991   
3   4   Arthur Silveira      574.908.123-05       Feminino  04/08/1952   
4   5  Vicente Teixeira      937.825.104-88  Não informado  04/02/1973   

   status externalCode  isEnriched        enrichedAt         enrichedBy  ...  \
0       1          NaN        True  04/08/2024 02:29  franciscocarvalho  ...   
1       2          NaN       False               NaN                NaN  ...   
2       1     UZZPQK51        True  31/10/2023 02:50           calebe40  ...   
3       1          NaN        True  07/07/2024 18:43        moraesluigi  ...   
4       1          NaN       False               NaN                NaN  ...   

  Recencia Frequencia

In [7]:
# Exportação dos datasets finais
try:
    df_customers_final.to_csv(
        'Fidelize_Customers_Enriquecido.csv',
        index=False,
        sep=';',
        decimal=',',
        encoding='utf-8-sig'
    )

    df_orders_cleaned.to_csv(
        'Fidelize_Orders_Limpo.csv',
        index=False,
        sep=';',
        decimal=',',
        encoding='utf-8-sig'
    )

    print("Arquivos 'Fidelize_Customers_Enriquecido.csv' e 'Fidelize_Orders_Limpo.csv' salvos com sucesso.")

except Exception as e:
    print(f"Erro ao salvar os arquivos: {e}")

Arquivos 'Fidelize_Customers_Enriquecido.csv' e 'Fidelize_Orders_Limpo.csv' salvos com sucesso.


In [8]:
# Visualização Exploratória

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")


# --- Gráfico 1: Distribuição de Clientes por Ciclo de Vida ---
df_ciclo = df_customers_final['CicloDeVida'].value_counts().reset_index()
fig_donut = px.pie(df_ciclo,
                   values='count',
                   names='CicloDeVida',
                   title='Distribuição de Clientes por Ciclo de Vida',
                   hole=0.4)
fig_donut.update_traces(textinfo='percent+label')
fig_donut.show()


# --- Gráfico 2: Receita Total por Ciclo de Vida (Barras) ---
df_receita_ciclo = df_customers_final.groupby('CicloDeVida')['ValorMonetario'].sum().reset_index()
fig_bar_ciclo = px.bar(df_receita_ciclo,
                     x='CicloDeVida',
                     y='ValorMonetario',
                     title='Receita Total (ValorMonetario) por Ciclo de Vida',
                     labels={'ValorMonetario': 'Receita Total', 'CicloDeVida': 'Ciclo de Vida'},
                     color='CicloDeVida')
fig_bar_ciclo.show()


# --- Gráfico 3: Segmentação RFM (Dispersão/Bolhas) ---
df_rfm_plot = df_customers_final.dropna(subset=['Recencia'])
fig_bubble_rfm = px.scatter(df_rfm_plot,
                          x='Recencia',
                          y='Frequencia',
                          size='ValorMonetario',
                          color='Segmento_RFM', # Usando a segmentação refinada
                          title='Análise de Clientes por RFM (Recência, Frequência, Valor)',
                          hover_name='name', # Mostra o nome do cliente
                          hover_data=['TicketMedio', 'CicloDeVida'],
                          size_max=60)
fig_bubble_rfm.update_layout(xaxis_title='Recência (Dias - Menor é Melhor)',
                             yaxis_title='Frequência (Pedidos - Maior é Melhor)')
fig_bubble_rfm.show()


# --- Gráfico 4: Contagem de Clientes por Gênero (Barras) ---
df_genero = df_customers_final['gender'].value_counts().reset_index()
fig_bar_gender = px.bar(df_genero,
                      x='gender',
                      y='count',
                      title='Distribuição de Clientes por Gênero',
                      labels={'count': 'Número de Clientes', 'gender': 'Gênero'},
                      color='gender')
fig_bar_gender.show()


# --- Gráfico 5: Receita ao Longo do Tempo (Linha) ---
df_orders_cleaned['createdAt'] = pd.to_datetime(df_orders_cleaned['createdAt'])
df_receita_tempo = df_orders_cleaned.set_index('createdAt').resample('ME')['totalAmount'].sum().reset_index()

fig_line_time = px.line(df_receita_tempo,
                        x='createdAt',
                        y='totalAmount',
                        title='Receita Total ao Longo do Tempo (por Mês)',
                        labels={'totalAmount': 'Receita Total', 'createdAt': 'Mês'},
                        markers=True)
fig_line_time.update_xaxes(dtick="M1", tickformat="%b %Y")
fig_line_time.show()